In [ ]:
import os
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- 1. SET UP YOUR SNOWFLAKE CREDENTIALS ---
conn = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database=os.getenv('SNOWFLAKE_DATABASE'),
    schema=os.getenv('SNOWFLAKE_SCHEMA'),
    role=os.getenv('SNOWFLAKE_ROLE')
)

# --- 2. DEFINE YOUR FOLDER PATH ---
folder_path = os.getenv('DATA_FOLDER_PATH')

print("Starting upload process...")

# --- 3. LOOP THROUGH EXCEL FILES AND UPLOAD ---
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)

        # Read the Excel file into a pandas DataFrame
        print(f"Reading {filename}...")
        df = pd.read_excel(file_path)

        # Clean up column names (Snowflake best practice: Uppercase, no spaces)
        df.columns = [str(c).upper().replace(' ', '_') for c in df.columns]

        # Use the filename (without the .xlsx extension) as the Snowflake table name
        table_name = os.path.splitext(filename)[0].upper()

        # Write the DataFrame to Snowflake
        # auto_create_table=True tells Snowflake to build the table if it doesn't exist
        print(f"Uploading to table: {table_name}...")
        success, nchunks, nrows, _ = write_pandas(
            conn=conn,
            df=df,
            table_name=table_name,
            auto_create_table=True
        )

        if success:
            print(f"Success! Inserted {nrows} rows into {table_name}.\n")
        else:
            print(f"Failed to upload {filename}.\n")

# Close the connection when done
conn.close()
print("All files processed and connection closed.")